## Deliverable 2. Create a Customer Travel Destinations Map.

In [40]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [41]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Esperance,AU,-33.87,121.90,57.20,82,90,5.82,overcast clouds
1,1,Atuona,PF,-9.80,-139.03,78.62,71,0,17.83,clear sky
2,2,Yar-Sale,RU,66.83,70.83,21.24,96,100,8.70,overcast clouds
3,3,Mmathubudukwane,BW,-24.60,26.43,77.54,33,78,8.43,broken clouds
4,4,Rikitea,PF,-23.12,-134.97,74.03,81,100,14.99,overcast clouds


In [42]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [43]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Atuona,PF,-9.80,-139.03,78.62,71,0,17.83,clear sky
3,3,Mmathubudukwane,BW,-24.60,26.43,77.54,33,78,8.43,broken clouds
4,4,Rikitea,PF,-23.12,-134.97,74.03,81,100,14.99,overcast clouds
9,9,Mahebourg,MU,-20.41,57.70,73.40,73,40,9.17,scattered clouds
10,10,Cabo San Lucas,MX,22.89,-109.91,87.80,58,5,10.29,clear sky
12,12,Mataura,NZ,-46.19,168.86,73.00,48,99,13.00,overcast clouds
14,14,Saint-Philippe,RE,-21.36,55.77,71.60,68,40,3.36,scattered clouds
15,15,Piranhas,BR,-16.43,-51.82,75.65,81,100,4.43,heavy intensity rain
16,16,Boyolangu,ID,-8.12,111.89,81.64,72,24,4.70,few clouds
20,20,Ulladulla,AU,-35.35,150.47,78.01,61,100,6.93,moderate rain


In [44]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                289
City                   289
Country                288
Lat                    289
Lng                    289
Max Temp               289
Humidity               289
Cloudiness             289
Wind Speed             289
Current Description    289
dtype: int64

In [45]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Atuona,PF,-9.80,-139.03,78.62,71,0,17.83,clear sky
3,3,Mmathubudukwane,BW,-24.60,26.43,77.54,33,78,8.43,broken clouds
4,4,Rikitea,PF,-23.12,-134.97,74.03,81,100,14.99,overcast clouds
9,9,Mahebourg,MU,-20.41,57.70,73.40,73,40,9.17,scattered clouds
10,10,Cabo San Lucas,MX,22.89,-109.91,87.80,58,5,10.29,clear sky
...,...,...,...,...,...,...,...,...,...,...
710,710,Hobyo,SO,5.35,48.53,81.07,77,100,11.01,overcast clouds
711,711,Galle,LK,6.04,80.22,79.03,85,53,9.95,light rain
714,714,Praia,CV,14.92,-23.51,78.80,78,40,17.22,scattered clouds
716,716,Oussouye,SN,12.48,-16.55,78.80,88,0,6.93,clear sky


In [46]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Atuona,PF,78.62,clear sky,-9.80,-139.03,
3,Mmathubudukwane,BW,77.54,broken clouds,-24.60,26.43,
4,Rikitea,PF,74.03,overcast clouds,-23.12,-134.97,
9,Mahebourg,MU,73.40,scattered clouds,-20.41,57.70,
10,Cabo San Lucas,MX,87.80,clear sky,22.89,-109.91,
12,Mataura,NZ,73.00,overcast clouds,-46.19,168.86,
14,Saint-Philippe,RE,71.60,scattered clouds,-21.36,55.77,
15,Piranhas,BR,75.65,heavy intensity rain,-16.43,-51.82,
16,Boyolangu,ID,81.64,few clouds,-8.12,111.89,
20,Ulladulla,AU,78.01,moderate rain,-35.35,150.47,


In [47]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")
    


Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [48]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Atuona,PF,78.62,clear sky,-9.80,-139.03,Villa Enata
3,Mmathubudukwane,BW,77.54,broken clouds,-24.60,26.43,Gill Sleepover Guesthouse
4,Rikitea,PF,74.03,overcast clouds,-23.12,-134.97,Pension Maro'i
9,Mahebourg,MU,73.40,scattered clouds,-20.41,57.70,Shandrani Beachcomber Resort & Spa
10,Cabo San Lucas,MX,87.80,clear sky,22.89,-109.91,Hotel Tesoro Los Cabos
...,...,...,...,...,...,...,...
710,Hobyo,SO,81.07,overcast clouds,5.35,48.53,Osmani Hotel
711,Galle,LK,79.03,light rain,6.04,80.22,Amangalla
714,Praia,CV,78.80,scattered clouds,14.92,-23.51,LT Aparthotel
716,Oussouye,SN,78.80,clear sky,12.48,-16.55,Auberge du Routard


In [49]:
# 8a. Create the output File (CSV)
hotel_output_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(hotel_output_file, index_label="City_ID")

In [50]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} & {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [51]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))